##colab上での操作時に使用

In [2]:
!pip install lightgbm

In [22]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    %cd /content/drive/MyDrive/MyApps/HorseRacingApp/app_azure_realtime/notebook
except:
    pass

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/MyApps/HorseRacingApp/app_azure_realtime/notebook


## import requred libraries

In [33]:
import pandas as pd
import numpy as np
from datetime import datetime
import pickle # 完成したデータを保存する用
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import TimeSeriesSplit, GroupShuffleSplit, train_test_split
from sklearn.preprocessing import LabelEncoder
import json
import matplotlib.pyplot as plt
import seaborn as sns
from optuna.integration import lightgbm as lgb

## Database

In [24]:
with open("df.json",mode="r") as f:
    df = json.load(f)
    df = pd.DataFrame(df)
df

,Hor_Num,Hor_name,Hor_sex_and_age,Horse_weight,Horse_weight_Flux,Jockey,JockeyWeight,Odds_popularity,Popularity,Race_Name,Race_Time,Race_date,Race_id,Ranking,Trainer_1,Trainer_2,Uni_Num,ato_3_F,competition_count,corner_rank_1,corner_rank_2,corner_rank_3,corner_rank_4,day_count,field_condition,field_length,field_type_1,field_type_2,hadicap,horse_class,horse_class_2,number_of_hourses,race_type,start_time,target,venue_area,weather
0,6,ウインルーア,牝2,438,4.0,横山武,54.0,16.0,3,2歳未勝利,109.7,1.595635e+12,202001010101,1,美浦,青木,6,35.6,1,5.0,5.0,3.0,3.0,1,良,1800,芝,右,馬齢,サラ系２歳,未勝利,6,[指],955,0,札幌,曇
1,2,アークライト,牡2,510,0.0,ルメー,54.0,1.9,2,2歳未勝利,110.0,1.595635e+12,202001010101,2,美浦,藤沢和,2,35.8,1,3.0,3.0,4.0,3.0,1,良,1800,芝,右,馬齢,サラ系２歳,未勝利,6,[指],955,0,札幌,曇
2,3,ギャラントウォリア,牡2,482,-6.0,池添,54.0,1.8,1,2歳未勝利,110.1,1.595635e+12,202001010101,3,栗東,平田,3,36.2,1,2.0,2.0,2.0,2.0,1,良,1800,芝,右,馬齢,サラ系２歳,未勝利,6,[指],955,0,札幌,曇
3,1,ジュンブーケ,牝2,442,0.0,亀田,52.0,22.2,4,2歳未勝利,110.5,1.595635e+12,202001010101,4,栗東,森,1,36.7,1,1.0,1.0,1.0,1.0,1,良,1800,芝,右,馬齢,サラ系２歳,未勝利,6,[指],955,1,札幌,曇
4,4,キタノマンゲツ,牡2,426,-8.0,藤岡康,54.0,55.7,5,2歳未勝利,111.0,1.595635e+12,202001010101,5,栗東,中尾,4,36.6,1,6.0,6.0,5.0,5.0,1,良,1800,芝,右,馬齢,サラ系２歳,未勝利,6,[指],955,1,札幌,曇
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87130,8,ダノンシュネラ,牝3,500,-4.0,岩田望,53.0,6.1,4,3歳以上2勝クラス,114.0,1.635552e+12,202109040712,6,栗東,池江,7,37.7,4,NaN,NaN,NaN,NaN,7,良,1800,ダ,右,定量,サラ系３歳以上,２勝クラス,10,(混),1610,1,阪神,晴
87131,10,メイショウシャガ,牡4,512,2.0,角田,54.0,94.6,9,3歳以上2勝クラス,114.1,1.635552e+12,202109040712,7,栗東,池添兼,8,37.5,4,NaN,NaN,NaN,NaN,7,良,1800,ダ,右,定量,サラ系３歳以上,２勝クラス,10,(混),1610,1,阪神,晴
87132,2,ブルーコンパクタ,牡5,474,4.0,和田竜,57.0,15.3,7,3歳以上2勝クラス,114.4,1.635552e+12,202109040712,8,栗東,大橋,2,37.4,4,NaN,NaN,NaN,NaN,7,良,1800,ダ,右,定量,サラ系３歳以上,２勝クラス,10,(混),1610,1,阪神,晴
87133,1,ルドンカズマ,牡4,496,0.0,松若,57.0,10.8,6,3歳以上2勝クラス,114.4,1.635552e+12,202109040712,9,栗東,杉山晴,1,37.1,4,NaN,NaN,NaN,NaN,7,良,1800,ダ,右,定量,サラ系３歳以上,２勝クラス,10,(混),1610,1,阪神,晴


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87135 entries, 0 to 87134
Data columns (total 37 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Hor_Num            87135 non-null  int64  
 1   Hor_name           87135 non-null  object 
 2   Hor_sex_and_age    87135 non-null  object 
 3   Horse_weight       87135 non-null  int64  
 4   Horse_weight_Flux  87121 non-null  float64
 5   Jockey             87135 non-null  object 
 6   JockeyWeight       87135 non-null  float64
 7   Odds_popularity    87135 non-null  float64
 8   Popularity         87135 non-null  int64  
 9   Race_Name          87135 non-null  object 
 10  Race_Time          87135 non-null  float64
 11  Race_date          87135 non-null  float64
 12  Race_id            87135 non-null  int64  
 13  Ranking            87135 non-null  int64  
 14  Trainer_1          87135 non-null  object 
 15  Trainer_2          87135 non-null  object 
 16  Uni_Num            871

In [26]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Hor_Num,87135.0,7.722259e+00,4.403047e+00,1.000000e+00,4.000000e+00,7.000000e+00,1.100000e+01,1.800000e+01
Horse_weight,87135.0,4.708984e+02,3.023449e+01,3.360000e+02,4.500000e+02,4.700000e+02,4.920000e+02,5.920000e+02
Horse_weight_Flux,87121.0,4.091895e-01,6.369148e+00,-4.200000e+01,-4.000000e+00,0.000000e+00,4.000000e+00,6.600000e+01
JockeyWeight,87135.0,5.492258e+01,1.800005e+00,4.800000e+01,5.400000e+01,5.500000e+01,5.600000e+01,6.400000e+01
Odds_popularity,87135.0,6.147556e+01,9.043596e+01,1.100000e+00,8.100000e+00,2.340000e+01,7.320000e+01,9.936000e+02
Popularity,87135.0,7.693464e+00,4.389745e+00,1.000000e+00,4.000000e+00,7.000000e+00,1.100000e+01,1.800000e+01
Race_Time,87135.0,1.025208e+02,2.942402e+01,5.420000e+01,8.040000e+01,1.003000e+02,1.169000e+02,3.181000e+02
Race_date,87135.0,1.606347e+12,1.664619e+10,1.578182e+12,1.592006e+12,1.606522e+12,1.620432e+12,1.635552e+12
Race_id,87135.0,2.020512e+11,4.976627e+07,2.020010e+11,2.020060e+11,2.020091e+11,2.021060e+11,2.021100e+11
Ranking,87135.0,7.667068e+00,4.378113e+00,1.000000e+00,4.000000e+00,7.000000e+00,1.100000e+01,1.800000e+01


## Add more features to the dataframe by computing some statistics

In [27]:
#Label encoding
le = LabelEncoder()
df['Race_id'] = le.fit_transform(df['Race_id'])
df.head()

,Hor_Num,Hor_name,Hor_sex_and_age,Horse_weight,Horse_weight_Flux,Jockey,JockeyWeight,Odds_popularity,Popularity,Race_Name,Race_Time,Race_date,Race_id,Ranking,Trainer_1,Trainer_2,Uni_Num,ato_3_F,competition_count,corner_rank_1,corner_rank_2,corner_rank_3,corner_rank_4,day_count,field_condition,field_length,field_type_1,field_type_2,hadicap,horse_class,horse_class_2,number_of_hourses,race_type,start_time,target,venue_area,weather
0,6,ウインルーア,牝2,438,4.0,横山武,54.0,16.0,3,2歳未勝利,109.7,1.595635e+12,0,1,美浦,青木,6,35.6,1,5.0,5.0,3.0,3.0,1,良,1800,芝,右,馬齢,サラ系２歳,未勝利,6,[指],955,0,札幌,曇
1,2,アークライト,牡2,510,0.0,ルメー,54.0,1.9,2,2歳未勝利,110.0,1.595635e+12,0,2,美浦,藤沢和,2,35.8,1,3.0,3.0,4.0,3.0,1,良,1800,芝,右,馬齢,サラ系２歳,未勝利,6,[指],955,0,札幌,曇
2,3,ギャラントウォリア,牡2,482,-6.0,池添,54.0,1.8,1,2歳未勝利,110.1,1.595635e+12,0,3,栗東,平田,3,36.2,1,2.0,2.0,2.0,2.0,1,良,1800,芝,右,馬齢,サラ系２歳,未勝利,6,[指],955,0,札幌,曇
3,1,ジュンブーケ,牝2,442,0.0,亀田,52.0,22.2,4,2歳未勝利,110.5,1.595635e+12,0,4,栗東,森,1,36.7,1,1.0,1.0,1.0,1.0,1,良,1800,芝,右,馬齢,サラ系２歳,未勝利,6,[指],955,1,札幌,曇
4,4,キタノマンゲツ,牡2,426,-8.0,藤岡康,54.0,55.7,5,2歳未勝利,111.0,1.595635e+12,0,5,栗東,中尾,4,36.6,1,6.0,6.0,5.0,5.0,1,良,1800,芝,右,馬齢,サラ系２歳,未勝利,6,[指],955,1,札幌,曇


In [28]:
"""馬別に統計量を取得"""
horse_df_1 = df[['Hor_name','Odds_popularity','Horse_weight','corner_rank_1','corner_rank_2', 'corner_rank_3', 'corner_rank_4','field_length','Horse_weight_Flux','JockeyWeight']].groupby('Hor_name').mean()
horse_df_1 = horse_df_1.sort_values('Hor_name')
horse_df_1.columns = [i+('_mean') for i in horse_df_1.columns]
horse_df_1 = horse_df_1.reset_index()
horse_df_2 = df[['Hor_name','Race_id']].groupby('Hor_name').count()
horse_df_2.columns = ['Race_experience_count']
horse_df_2 = horse_df_2.reset_index()

"""## 騎手別に統計量を取得"""
## As it is

"""## フィールドデータなどの取得"""
## As it is

"""##各dfを結合して特徴量を作成する"""
df = df.merge(horse_df_1,how='left',on='Hor_name')
df = df.merge(horse_df_2,how='left',on='Hor_name')
df = df.sort_values('Race_id')

"""カテゴリ変数をlabel encodingで変換する"""
cat_cols = ['Race_Name', 'Hor_name','Race_date', 'Hor_sex_and_age', 'Jockey', 'Trainer_1', 'Trainer_2', 'weather', 'field_condition', 'venue_area', 'horse_class', 'horse_class_2', 'race_type', 'hadicap', 'field_type_1', 'field_type_2']
for col in cat_cols:
    le = LabelEncoder()
    le.fit(df[col])
    df[col] = le.transform(df[col])

#最終的な特徴量
df.head()   

,Hor_Num,Hor_name,Hor_sex_and_age,Horse_weight,Horse_weight_Flux,Jockey,JockeyWeight,Odds_popularity,Popularity,Race_Name,Race_Time,Race_date,Race_id,Ranking,Trainer_1,Trainer_2,Uni_Num,ato_3_F,competition_count,corner_rank_1,corner_rank_2,corner_rank_3,corner_rank_4,day_count,field_condition,field_length,field_type_1,field_type_2,hadicap,horse_class,horse_class_2,number_of_hourses,race_type,start_time,target,venue_area,weather,Odds_popularity_mean,Horse_weight_mean,corner_rank_1_mean,corner_rank_2_mean,corner_rank_3_mean,corner_rank_4_mean,field_length_mean,Horse_weight_Flux_mean,JockeyWeight_mean,Race_experience_count
0,6,1352,10,438,4.0,148,54.0,16.0,3,2,109.7,61,0,1,3,225,6,35.6,1,5.0,5.0,3.0,3.0,1,2,1800,1,0,3,0,2,6,16,955,0,6,3,41.762500,445.000000,6.750000,6.875000,6.000000,6.714286,1775.000000,4.000000,53.375000,8
1,2,872,20,510,0.0,7,54.0,1.9,2,2,110.0,61,0,2,3,203,2,35.8,1,3.0,3.0,4.0,3.0,1,2,1800,1,0,3,0,2,6,16,955,0,6,3,1.987500,517.500000,5.500000,4.750000,5.333333,5.666667,1562.500000,1.000000,55.375000,8
2,3,3041,20,482,-6.0,160,54.0,1.8,1,2,110.1,61,0,3,1,87,3,36.2,1,2.0,2.0,2.0,2.0,1,2,1800,1,0,3,0,2,6,16,955,0,6,3,11.090000,493.000000,2.400000,2.300000,2.500000,2.500000,1860.000000,1.000000,55.000000,10
3,1,5796,10,442,0.0,22,52.0,22.2,4,2,110.5,61,0,4,1,127,1,36.7,1,1.0,1.0,1.0,1.0,1,2,1800,1,0,3,0,2,6,16,955,1,6,3,31.766667,442.666667,1.833333,2.000000,1.333333,1.333333,1666.666667,0.000000,52.000000,6
4,4,2777,20,426,-8.0,216,54.0,55.7,5,2,111.0,61,0,5,1,4,4,36.6,1,6.0,6.0,5.0,5.0,1,2,1800,1,0,3,0,2,6,16,955,1,6,3,133.200000,437.666667,8.000000,8.333333,6.000000,6.333333,1683.333333,1.666667,53.333333,6


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87135 entries, 0 to 80946
Data columns (total 47 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Hor_Num                 87135 non-null  int64  
 1   Hor_name                87135 non-null  int64  
 2   Hor_sex_and_age         87135 non-null  int64  
 3   Horse_weight            87135 non-null  int64  
 4   Horse_weight_Flux       87121 non-null  float64
 5   Jockey                  87135 non-null  int64  
 6   JockeyWeight            87135 non-null  float64
 7   Odds_popularity         87135 non-null  float64
 8   Popularity              87135 non-null  int64  
 9   Race_Name               87135 non-null  int64  
 10  Race_Time               87135 non-null  float64
 11  Race_date               87135 non-null  int64  
 12  Race_id                 87135 non-null  int64  
 13  Ranking                 87135 non-null  int64  
 14  Trainer_1               87135 non-null

##Training with cross-validation

In [36]:
# データの並び順を元に分割する
folds = TimeSeriesSplit(n_splits=5)

X = df.drop(['Ranking','target','Race_Time','corner_rank_1',
    'corner_rank_2', 'corner_rank_3', 'corner_rank_4','target'],axis=1) #Leakageになりそうなものも含めて除外する
y = df['Ranking']

X_train = X[~-20000:]
X_val = X[-20000:]
y_train = y[~-20000:]
y_val = y[-20000:]

#評価結果格納用

params = {
'task':'train',
'boosting_type':'gbdt',
'objective':'lambdarank',
'metric':'ndcg',
'nucg_eval_at':[5]
}
best_params={}

train_data = pd.concat([X_train,y_train],axis=1)
train_groups = train_data.groupby('Race_id').size().to_frame('size')['size'].to_numpy()
val_data = pd.concat([X_val,y_val],axis=1)
val_groups = val_data.groupby('Race_id').size().to_frame('size')['size'].to_numpy()


train_dataset = lgb.Dataset(X_train,y_train,group=train_groups)
val_dataset = lgb.Dataset(X_val, y_val, reference=train_dataset, group=val_groups)


model = lgb.LightGBMTuner(params=params,
                                train_set=train_dataset,
                                valid_sets=[train_dataset,val_dataset],
                                num_boost_round=5000,
                                verbose_eval=100,
                                early_stopping_round
                                # folds=folds.split(X,groups=df['Race_id'])
                            )            
model.run()

best_params = model.best_params

print(best_params)
# for i, (train_index, val_index) in enumerate(folds.split(df,groups=df['Race_id'])):
#     print(f"Fold:{i+1} has started." )
#     X_train, X_val = X.iloc[train_index], X.iloc[val_index]
#     y_train, y_val = y.iloc[train_index], y.iloc[val_index]

#     #group引数用
#     train_data = pd.concat([X_train,y_train],axis=1)
#     train_groups = train_data.groupby('Race_id').size().to_frame('size')['size'].to_numpy()
#     val_data = pd.concat([X_val,y_val],axis=1)
#     val_groups = val_data.groupby('Race_id').size().to_frame('size')['size'].to_numpy()
#     # print(train_groups)
#     # print(val_groups)

#     train_dataset = lgb.Dataset(X_train, y_train, group=train_groups)
#     val_dataset = lgb.Dataset(X_val, y_val, reference=train_dataset, group=val_groups)

#     params = {
#     'task': 'train',
#     'boosting_type': 'gbdt',
#     'objective': 'lambdarank',
#     'metric':'ndcg'
#     }
#     best_params={}

#     model = lgb.train(params = params,
#                     train_set=train_dataset,
#                     valid_sets= [train_dataset,val_dataset],
#                     num_boost_round =5000,
#                     early_stopping_rounds=50,
#                     verbose_eval = 50,
#                     )
#     oof_predictions[val_index] = model.predict(X_val,num_iteration=model.best_iteration)
    
#     with open(f'pred_model_{i}.pkl', mode='wb') as f: #save each model
#         pickle.dump(model,f)

# result = pd.DataFrame(oof_predictions,columns=['predicted_class'])

# df_result = pd.concat([df,result],axis=1)

[I 2021-11-14 08:41:10,924] A new study created in memory with name: no-name-ee410f67-4e92-4c86-b020-087bd4bc31e6








  0%|          | 0/7 [00:00<?, ?it/s]







feature_fraction, val_score: -inf:   0%|          | 0/7 [00:00<?, ?it/s]

[100]	valid_0's ndcg@1: 0.875319	valid_0's ndcg@2: 0.872605	valid_0's ndcg@3: 0.881355	valid_0's ndcg@4: 0.893116	valid_0's ndcg@5: 0.905064	valid_1's ndcg@1: 0.873221	valid_1's ndcg@2: 0.878573	valid_1's ndcg@3: 0.885846	valid_1's ndcg@4: 0.897695	valid_1's ndcg@5: 0.908725
[200]	valid_0's ndcg@1: 0.911128	valid_0's ndcg@2: 0.907533	valid_0's ndcg@3: 0.909943	valid_0's ndcg@4: 0.919135	valid_0's ndcg@5: 0.927835	valid_1's ndcg@1: 0.914123	valid_1's ndcg@2: 0.91463	valid_1's ndcg@3: 0.913687	valid_1's ndcg@4: 0.922839	valid_1's ndcg@5: 0.931651
[300]	valid_0's ndcg@1: 0.929838	valid_0's ndcg@2: 0.924846	valid_0's ndcg@3: 0.924968	valid_0's ndcg@4: 0.932183	valid_0's ndcg@5: 0.93909	valid_1's ndcg@1: 0.93369	valid_1's ndcg@2: 0.929868	valid_1's ndcg@3: 0.926993	valid_1's ndcg@4: 0.934666	valid_1's ndcg@5: 0.941155
[400]	valid_0's ndcg@1: 0.946086	valid_0's ndcg@2: 0.938296	valid_0's ndcg@3: 0.936279	valid_0's ndcg@4: 0.942042	valid_0's ndcg@5: 0.948081	valid_1's ndcg@1: 0.951136	valid_1









feature_fraction, val_score: 0.986359:   0%|          | 0/7 [00:17<?, ?it/s]







feature_fraction, val_score: 0.986359:  14%|#4        | 1/7 [00:17<01:44, 17.39s/it][I 2021-11-14 08:41:28,332] Trial 0 finished with value: 0.986358574975371 and parameters: {'feature_fraction': 1.0}. Best is trial 0 with value: 0.986358574975371.








feature_fraction, val_score: 0.986359:  14%|#4        | 1/7 [00:17<01:44, 17.39s/it]

[1000]	valid_0's ndcg@1: 0.98416	valid_0's ndcg@2: 0.974543	valid_0's ndcg@3: 0.969243	valid_0's ndcg@4: 0.970128	valid_0's ndcg@5: 0.972789	valid_1's ndcg@1: 0.986359	valid_1's ndcg@2: 0.975018	valid_1's ndcg@3: 0.96965	valid_1's ndcg@4: 0.969947	valid_1's ndcg@5: 0.973641
[100]	valid_0's ndcg@1: 0.826064	valid_0's ndcg@2: 0.826559	valid_0's ndcg@3: 0.837227	valid_0's ndcg@4: 0.85263	valid_0's ndcg@5: 0.86691	valid_1's ndcg@1: 0.831404	valid_1's ndcg@2: 0.834954	valid_1's ndcg@3: 0.843907	valid_1's ndcg@4: 0.859835	valid_1's ndcg@5: 0.872229
[200]	valid_0's ndcg@1: 0.895455	valid_0's ndcg@2: 0.889196	valid_0's ndcg@3: 0.892797	valid_0's ndcg@4: 0.903555	valid_0's ndcg@5: 0.913524	valid_1's ndcg@1: 0.90278	valid_1's ndcg@2: 0.896679	valid_1's ndcg@3: 0.898316	valid_1's ndcg@4: 0.908889	valid_1's ndcg@5: 0.9181
[300]	valid_0's ndcg@1: 0.92329	valid_0's ndcg@2: 0.916327	valid_0's ndcg@3: 0.915757	valid_0's ndcg@4: 0.923414	valid_0's ndcg@5: 0.931952	valid_1's ndcg@1: 0.933996	valid_1's n









feature_fraction, val_score: 0.986359:  14%|#4        | 1/7 [00:29<01:44, 17.39s/it]







feature_fraction, val_score: 0.986359:  29%|##8       | 2/7 [00:30<01:12, 14.58s/it][I 2021-11-14 08:41:40,944] Trial 1 finished with value: 0.9848550181639871 and parameters: {'feature_fraction': 0.4}. Best is trial 0 with value: 0.986358574975371.








feature_fraction, val_score: 0.986359:  29%|##8       | 2/7 [00:30<01:12, 14.58s/it]

...

[100]	valid_0's ndcg@1: 0.980449	valid_0's ndcg@2: 0.967246	valid_0's ndcg@3: 0.960486	valid_0's ndcg@4: 0.961022	valid_0's ndcg@5: 0.964628	valid_1's ndcg@1: 0.98151	valid_1's ndcg@2: 0.965816	valid_1's ndcg@3: 0.959447	valid_1's ndcg@4: 0.960078	valid_1's ndcg@5: 0.964475
[200]	valid_0's ndcg@1: 0.99785	valid_0's ndcg@2: 0.99064	valid_0's ndcg@3: 0.984363	valid_0's ndcg@4: 0.982748	valid_0's ndcg@5: 0.983562	valid_1's ndcg@1: 0.997486	valid_1's ndcg@2: 0.989748	valid_1's ndcg@3: 0.98436	valid_1's ndcg@4: 0.98307	valid_1's ndcg@5: 0.983798
[300]	valid_0's ndcg@1: 0.999898	valid_0's ndcg@2: 0.997073	valid_0's ndcg@3: 0.992627	valid_0's ndcg@4: 0.990236	valid_0's ndcg@5: 0.990395	valid_1's ndcg@1: 0.999666	valid_1's ndcg@2: 0.996375	valid_1's ndcg@3: 0.992406	valid_1's ndcg@4: 0.990256	valid_1's ndcg@5: 0.990666
[400]	valid_0's ndcg@1: 1	valid_0's ndcg@2: 0.999092	valid_0's ndcg@3: 0.996555	valid_0's ndcg@4: 0.99416	valid_0's ndcg@5: 0.993775	valid_1's ndcg@1: 1	valid_1's ndcg@2: 0.9987









min_data_in_leaf, val_score: 1.000000:  60%|######    | 3/5 [02:13<01:05, 32.57s/it]







min_data_in_leaf, val_score: 1.000000:  80%|########  | 4/5 [02:13<00:33, 33.66s/it][I 2021-11-14 09:12:22,129] Trial 66 finished with value: 1.0 and parameters: {'min_child_samples': 5}. Best is trial 63 with value: 1.0.








min_data_in_leaf, val_score: 1.000000:  80%|########  | 4/5 [02:13<00:33, 33.66s/it]

[100]	valid_0's ndcg@1: 0.98101	valid_0's ndcg@2: 0.967422	valid_0's ndcg@3: 0.960949	valid_0's ndcg@4: 0.961451	valid_0's ndcg@5: 0.964902	valid_1's ndcg@1: 0.982088	valid_1's ndcg@2: 0.968168	valid_1's ndcg@3: 0.960295	valid_1's ndcg@4: 0.96116	valid_1's ndcg@5: 0.965177
[200]	valid_0's ndcg@1: 0.99785	valid_0's ndcg@2: 0.989927	valid_0's ndcg@3: 0.984141	valid_0's ndcg@4: 0.982534	valid_0's ndcg@5: 0.983538	valid_1's ndcg@1: 0.998657	valid_1's ndcg@2: 0.988799	valid_1's ndcg@3: 0.983812	valid_1's ndcg@4: 0.982385	valid_1's ndcg@5: 0.983873
[300]	valid_0's ndcg@1: 0.999591	valid_0's ndcg@2: 0.996276	valid_0's ndcg@3: 0.992433	valid_0's ndcg@4: 0.989997	valid_0's ndcg@5: 0.990211	valid_1's ndcg@1: 0.999666	valid_1's ndcg@2: 0.995493	valid_1's ndcg@3: 0.99253	valid_1's ndcg@4: 0.99011	valid_1's ndcg@5: 0.990496
[400]	valid_0's ndcg@1: 1	valid_0's ndcg@2: 0.999092	valid_0's ndcg@3: 0.996523	valid_0's ndcg@4: 0.99422	valid_0's ndcg@5: 0.993872	valid_1's ndcg@1: 1	valid_1's ndcg@2: 0.9986









min_data_in_leaf, val_score: 1.000000:  80%|########  | 4/5 [02:47<00:33, 33.66s/it]







min_data_in_leaf, val_score: 1.000000: 100%|##########| 5/5 [02:47<00:00, 33.49s/it]

{'task': 'train', 'boosting_type': 'gbdt', 'objective': 'lambdarank', 'metric': 'ndcg', 'nucg_eval_at': [5], 'feature_pre_filter': False, 'lambda_l1': 0.0, 'lambda_l2': 0.0, 'num_leaves': 229, 'feature_fraction': 0.7, 'bagging_fraction': 1.0, 'bagging_freq': 0, 'min_child_samples': 20}


In [ ]:
pd.options.display.max_rows=10000
df_result[['Ranking','predicted_class']].head(10000)